In [1]:
import socket
import time
import threading
import dash
from dash import Dash, html, dcc, Input, Output, State, callback_context
import plotly.graph_objects as go
from collections import deque
import dash_daq as daq

message_str_list = []
response_str_list = []
X = deque(maxlen=20)
X.append(1)
Y = deque(maxlen=20)
Y.append(1)



monitor_thread_running = False
client_socket = None

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    id="ust-div",
    children=[
        html.Div(
            id="ust-div-icerik",
            children=[
                html.Div([
                    html.Label("IP ADRESİ", style={'color': "#fec036", "margin-left": "25px", "margin-bottom": "10px"}),
                    dcc.Input(id='ip-input', type='text', value='', style={'color': "#fec036", 'height': '25px', 'width': '150px', "margin-left": "25px", "margin-right": "40px"}),
                    html.Button('Connect', id='connect-button', n_clicks=0, style={"color": "#fec036",'height': '30px', 'width': '250px', "margin-left": "25px", "margin-top": "20px"})
                ], style={'background-color': '#2b2b2b', 'width': '33.33%', "height": "100%", 'display': 'inline-block'}),
                html.Div([
                    html.Label("PORT", style={'color': "#fec036", "margin-left": "25px", "margin-bottom": "10px"}),
                    dcc.Input(id='port-input', type='text', value='', style={'color': "#fec036", 'height': '25px', 'width': '150px', "margin-left": "25px", "margin-right": "40px"}),
                    html.Button('Disconnect', id='disconnect-button', n_clicks=0, style={"color": "#fec036",'height': '30px', 'width': '250px', "margin-left": "25px", "margin-top": "20px"})
                ], style={'background-color': "#2b2b2b", 'width': '33.33%', "height": "100%", 'display': 'inline-block'}),
                html.Div(id='output-state')
                
            ],
            style={"background-color": "#2b2b2b", "width": "100%", "height": "200px"}
        ),
        html.Div(
            id="alt-div",
            children=[
                html.Div([
                    html.Div(
                        id="small-div-85",
                        children=[
                            html.Pre(id='monitor-read-display', style={"white-space": "pre-wrap", "word-wrap": "break-word"}),
                        ],
                        style={
                            "width": "815.12", "height": "100%", "background-color": "red",
                            "overflow": "auto",
                            "background-color": "#2b2b2b",
                            "color": "#fec036"
                        }
                    ),
                ], style={'background-color': "#2b2b2b", 'width': '30%', "height": "100%", 'display': 'inline-block'}),
                html.Div([
                    dcc.Graph(id='live-graph', animate=True, style={"width": "800px", "height": "400px"}),
                    dcc.Interval(
                        id='graph-update',
                        interval=1 * 1000
                    ),
                ], style={'background-color': "#2b2b2b", 'width': '70%', "height": "100%", 'display': 'inline-block'})
            ],
            style={"background-color": "#2b2b2b", "width": "100%", "height": "400px"}
        ),
    ],
    style={"background-color": "#2b2b2b", "width": "100%", "height": "100%"}
)









def message(client_socket, addr):
    global monitor_thread_running
    message_str = "READ*{}".format(addr)
    message_str_list.append(message_str)

    if not monitor_thread_running:
        start_monitor_thread()

    client_socket.sendall(message_str.encode())

    response = client_socket.recv(1024).decode()
    response_str_list.append(response)

    run_monitor_thread()

    return response





def monitor_thread():
    while True:
        time.sleep(5)


def start_monitor_thread():
    global monitor_thread_running
    monitor_thread_running = True
    monitor = threading.Thread(target=monitor_thread)
    monitor.daemon = True
    monitor.start()


def run_monitor_thread():
    if not monitor_thread_running:
        start_monitor_thread()
    else:
        monitor = threading.Thread(target=monitor_thread)
        monitor.daemon = True
        monitor.start()


def main(server_ip, server_port):
    global client_socket
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
            client_socket.connect((server_ip, server_port))
            print("Connected to the server.")
            
            while True:
                time.sleep(1)
                response1 = message(client_socket, 23)
                

                time.sleep(1)
                response2 = message(client_socket, 23)
                
                time.sleep(1)
                response3 = message(client_socket, 23)
                
                time.sleep(1)



    except ConnectionRefusedError:
        print("Connection to the server was refused. The server may not be running.")
    except OSError as e:
        if e.errno == 11001:
            print("Error resolving the server's IP address or port number.")
        else:
            print(f"An unexpected error occurred: {e}")






@app.callback(
    [Output('output-state', 'children')],
    [Input('connect-button', 'n_clicks'),
     Input('disconnect-button', 'n_clicks')],
    [State('ip-input', 'value'),
     State('port-input', 'value')]
)
def send_command_to_server(connect_clicks, disconnect_clicks, ip_value, port_value):
    ctx = callback_context
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    style = {'color': '#fec036'}

    if button_id == 'connect-button' and connect_clicks is not None and connect_clicks > 0:
        if ip_value and port_value:
            port_value_sck = int(port_value)
            threading.Thread(target=main, args=(ip_value, port_value_sck)).start()
            return ("Connection established.")
        else:
            return ("Please enter IP address and port number.")
    elif button_id == 'disconnect-button' and disconnect_clicks is not None and disconnect_clicks > 0:
        client_socket.close()
        return ("Connection terminated.",)
    return (None,)




@app.callback(
    Output('live-graph', 'figure'),
    [Input('graph-update', 'n_intervals')]
)
def update_graph_scatter(n):
    global X
    global Y
    X.append(X[-1] + 1)
    Y.append(response_str_list[-1])
    trace1 = go.Scatter(
        x=list(X),
        y=list(Y),
        name='temperature_value',
        mode='lines+markers'
    )
    layout = go.Layout(xaxis=dict(range=[min(X), max(X)]),
                       yaxis=dict(range=[0, 60]))
    return {'data': [trace1], 'layout': layout}


@app.callback(
    Output('monitor-read-display', 'children'),
    [Input('graph-update', 'n_intervals')]
)
def update_monitor_read_display(n):
    if message_str_list and response_str_list:
        combined_text = ""
        for message, response in zip(message_str_list, response_str_list):
            combined_text += f"Sent message: {message}\nReceived message: {response}\n"
        return combined_text
    else:
        return "No data"



def dash_server():
    app.run_server( port=6921)



if __name__ == "__main__":
    dash_thread = threading.Thread(target=dash_server)
    dash_thread.start()


Dash is running on http://127.0.0.1:6921/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:6921/ (Press CTRL+C to quit)
127.0.0.1 - - [14/May/2024 00:35:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:45] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:45] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\muham\anaconda3\lib\site-packages\dash\dash.py", line 1372, in dispatch
    response.set_d

127.0.0.1 - - [14/May/2024 00:35:45] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [14/May/2024 00:35:46] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\muham\anaconda3\lib\site-packages\dash\dash.py", line 1372, in dispatch
    response.set_d

...

127.0.0.1 - - [14/May/2024 00:35:48] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\muham\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\muham\anaconda3\lib\site-packages\dash\dash.py", line 1372, in dispatch
    response.set_d

127.0.0.1 - - [14/May/2024 00:35:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:48] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [14/May/2024 00:35:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:35:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [14/May/2024 00:36:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:36:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:36:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:36:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:36:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:36:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:36:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:36:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:36:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:36:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:36:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:36:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [14/May/2024 00:37:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:37:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:37:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:37:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:37:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:37:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:37:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:37:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:37:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:37:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:37:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 00:37:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.